# Download dos casos de Malária

#### Os casos de Malária foram baixados a partir do repositório do Sistema de Informação de Agravos de Notificação (SINAN),  que é alimentado, principalmente, pela notificação e investigação de casos de doenças e agravos que constam da lista nacional de doenças de notificação compulsória.  O downlod foi feito a partir do Package Python PySUS, criado para auxiliar no acesso e manipulação de base de dados publicadas pelo DATASUS.

#### Maiores informações em:

 - [SINAN] (https://portalsinan.saude.gov.br/sinan-net)
 - [Malária] (https://portalsinan.saude.gov.br/malaria)
 - [PySUS] (https://pysus.readthedocs.io/en/latest/)


#### 1 - Importando o método SINAN do pacote PySUS

In [1]:
from pysus.ftp.databases.sinan import SINAN

#### 2 - Carregando os dados a partir do servidor do DataSUS

In [2]:
sinan = SINAN().load()

#### 3 - Lista os arquivos disponíveis para Malária

In [3]:
arquivos = sinan.get_files(dis_code=["MALA"])

#### 4 - Download dos arquivos disponíveis

In [4]:
parquets = sinan.download(arquivos)

100%|██████████| 115k/115k [00:00<00:00, 65.4MB/s]


#### 5 - Converte os arquivos parquet em uma lista de dataframes

In [5]:
dataframes = [parquet.to_dataframe() for parquet in parquets]

#### 6 - Concatena em um único dataframe selecionando apenas as colunas de interesse

In [6]:
import pandas as pd

In [7]:
selected_columns = ['DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'ID_MUNICIP']
selected_dfs = [df[selected_columns] for df in dataframes]
df = pd.concat(selected_dfs, ignore_index=True, axis=0)

#### 7 - Corrige os tipos dos campos de data e numéricos, e decodifica e o campo da semana da notificação

In [8]:
import numpy as np

In [9]:
# Cria a função convert_week para decodificar a coluna "SEM_NOT" 
def convert_week(x):
        try:
            w = int(x) % 100
        except ValueError:
            w = np.nan
        return w


In [10]:
    for cname in df.columns:
        df[cname].replace("", np.nan, inplace=True)
        if cname.startswith(("NU", "ID")):
            try:
                df[cname] = pd.to_numeric(df[cname])
            except ValueError as e:
                print(f"Column {cname} could not be converted to numeric: {e}")
                # certain IDs can be alphanumerical
                pass
        elif cname.startswith("DT"):
            try:
                df[cname] = pd.to_datetime(df[cname])
            except ValueError as e:
                print(f"Column {cname} could not be converted to date: {e}")
        elif cname.startswith("SEM"):
            df[cname] = df[cname].map(convert_week)

/tmp/ipykernel_738/711638361.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[cname].replace("", np.nan, inplace=True)
/tmp/ipykernel_738/711638361.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df

#### 8 - Agrupa e contando os registros por data e município

In [11]:
counts = df.groupby(['DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'ID_MUNICIP']).size().reset_index(name='count')

#### 9 - Ligação com a tabela de municípios importada com o pacote Geobr
#### Até o ano de 2006, o código do município apresenta 7 dígitos, depois de 2006 o código do município apresenta 6 dígitos. Desta maneira o merge será dividido em duas etapas

In [12]:
from geobr import read_municipality

In [13]:
mun = read_municipality(code_muni="all", year=2022)

In [14]:
# Converte a coluna 'code_muni' de float para int em uma nova coluna 'ID_MUNICIP'
mun['ID_MUNICIP_1'] = mun['code_muni'].astype('Int64')

# Remover o último dígito da coluna 'ID_MUNICIP'
mun['ID_MUNICIP_2'] = mun['ID_MUNICIP_1'].astype(str).str[:-1].astype(int)

In [15]:
selecao_1 = (counts['DT_NOTIFIC'] >= '2004-01-01') & (counts['DT_NOTIFIC'] <= '2006-12-31')
counts_1 = counts[selecao_1]


In [16]:
selecao_2 = (counts['DT_NOTIFIC'] >= '2007-01-01') & (counts['DT_NOTIFIC'] <= '2023-12-31')
counts_2 = counts[selecao_2]


In [17]:
# Junção dos dataframes 'counts' e 'mun' mantendo todos os registros mesmo quando não houver correspondência
merged_1 = counts_1.merge(mun, left_on='ID_MUNICIP', right_on='ID_MUNICIP_1', how='outer', indicator=True)
merged_2 = counts_2.merge(mun, left_on='ID_MUNICIP', right_on='ID_MUNICIP_2', how='outer', indicator=True)

In [18]:
merged = pd.concat([merged_1, merged_2])

#### 10 - Exportando em CSV

In [19]:
# Reordenando as colunas
sinan_malaria_2004_2023 = merged[['DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'code_muni', 'name_muni', 'abbrev_state', 'name_state', 'name_region', 'count', 'geometry']]

# Renomeando as colunas
sinan_malaria_2004_2023.columns = ['data', 'semana', 'ano','geocodigo', 'municipio',  'uf_sigla', 'uf', 'regiao', 'casos', 'geometry']


In [20]:
# Exportando para CSV
sinan_malaria_2004_2023.to_csv('sinan_malaria_2004_2023.csv',sep=';')

#### 11 - Exportando em GeoJason

In [32]:
# Importando geopandas
import geopandas as gpd

In [33]:
# Convertendo para GeoDataFrame
gdf = gpd.GeoDataFrame(sinan_malaria_2004_2023)

In [34]:
# Exportanod para GeoJSON
gdf.to_file('sinan_malaria_2004_2023.geojson', driver='GeoJSON')